## Includes

In [1]:
import sys
import os
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config


config_info = {
  "canalyst_api_key": "",
   "s3_access_key_id": "",
    "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}


config = Config(config=config_info)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

A configuration file has been created for you in 
/Users/jedgore/canalyst/keys.json.


# Example: Characteristics of a Successful Stock in a Sector
## Workflow:
<li>Set switches
<li>Create target vector = "earnings sharpe ratio" or alpha post-EPS / standard deviation post-EPS
<li>Create ModelSet for target list
<li>Find common names
<li>Create a model_frame of common names
<li>Regress against target vector
<li>List coefficients

## Control Switches: 
### Change ticker list as needed etc

In [2]:
yahoo_tickers = ['TJX',
'LULU',
'ROST',
'BURL',
'RL',
'GPS',
'LEVI',
'TPR',
'ZUMZ',
'CPRI',
'FL',
'CRI',
'AEO',
'URBN',
'BKE',
'GES',
'ANF']
target_list = ['TJX US',
'LULU US',
'ROST US',
'BURL US',
'RL US',
'GPS US',
'LEVI US',
'TPR US',
'ZUMZ US',
'CPRI US',
'FL US',
'CRI US',
'AEO US',
'URBN US',
'BKE US',
'GES US',
'ANF US']
index_ticker = "^GSPC"

### Get price data and calculate alpha 

In [3]:
price_list = []
for yahoo_ticker in yahoo_tickers:
    print(yahoo_ticker)
    df = cdr.get_earnings_and_prices(ticker=yahoo_ticker,index_ticker = index_ticker) 
    price_list.append(df)

TJX
LULU
ROST
BURL
RL
GPS
LEVI
TPR
ZUMZ
CPRI
FL
CRI
AEO
URBN
BKE
GES
ANF


In [4]:
pd.concat(price_list)[['ticker','alpha_5_day','epssurprisepct']].groupby('ticker').describe().reset_index().sort_values([('alpha_5_day','mean')],ascending=False)

ticker alpha_5_day                                        epssurprisepct  \
                count  mean  std   min   25%   50%  75%  max          count   
7     GES       49.00  0.03 0.19 -0.22 -0.08  0.01 0.08 1.04          53.00   
0     AEO       49.00  0.01 0.06 -0.12 -0.02  0.02 0.04 0.19          53.00   
6      FL       49.00  0.01 0.06 -0.17 -0.02  0.01 0.04 0.19          53.00   
10   LULU       49.00  0.01 0.08 -0.25 -0.05  0.01 0.07 0.15          53.00   
2     BKE       49.00  0.01 0.05 -0.07 -0.03  0.01 0.03 0.15          53.00   
12   ROST       49.00  0.01 0.05 -0.08 -0.02  0.01 0.03 0.14          53.00   
1     ANF       49.00  0.00 0.07 -0.12 -0.05  0.01 0.05 0.20          53.00   
3    BURL       30.00  0.00 0.04 -0.07 -0.01  0.01 0.02 0.09          34.00   
14    TPR       49.00  0.00 0.04 -0.08 -0.02  0.00 0.03 0.11          53.00   
4    CPRI       37.00  0.00 0.05 -0.10 -0.02 -0.00 0.04 0.10          41.00   
16   ZUMZ       49.00  0.00 0.11 -0.28 -0.06 -0.01 0.06 0.28          53.00   
13    TJX       49.00  0.00 0.03 -0.07 -0.01  0.01 0.02 0.07          53.00   
5     CRI       50.00 -0.00 0.04 -0.14 -0.02 -0.00 0.03 0.08          53.00   
15   URBN       49.00 -0.00 0.10 -0.22 -0.07  0.01 0.08 0.19          53.00   
11     RL       49.00 -0.00 0.04 -0.09 -0.03 -0.00 0.02 0.09          53.00   
9    LEVI        8.00 -0.00 0.08 -0.13 -0.03  0.01 0.04 0.08          11.00   
8     GPS       49.00 -0.02 0.08 -0.25 -0.05  0.00 0.03 0.10          53.00   

                                                       
     mean     std      min   25%   50%   75%      max  
7   45.00  146.82   -87.56 -0.32 13.56 36.74  1037.25  
0   94.64  477.52  -182.83  0.00  2.63  9.09  2900.00  
6   12.55   42.86  -169.08  2.47  7.14 16.98   119.03  
10  12.30   12.51    -6.17  4.31  7.99 15.68    44.02  
2   10.86  216.04  -988.89 -2.59  3.00  6.49  1190.91  
12 -47.10  400.89 -2906.45  0.10  1.99  6.00   122.56  
1  318.69 2099.23  -137.03 -0.28 10.15 47.19 15300.00  
3   19.26   57.16  -207.49  4.37 13.17 32.11   213.56  
14   8.21   30.90  -134.78  1.83  4.43 11.56   156.64  
4  123.72  443.51   -24.14  5.56 15.35 39.53  2095.12  
16  81.22  358.66   -55.84  5.59 15.05 31.37  2610.53  
13  -2.88   47.35  -315.73  0.00  2.02  4.22    79.29  
5   29.03  109.94  -431.97  4.02 11.22 25.42   526.58  
15   3.60   80.09  -487.50 -1.90  5.17 11.66   208.57  
11  21.05   36.16   -69.15  6.43 10.32 22.11   160.23  
9   42.99   51.76     2.44 12.29 28.00 35.77   152.75  
8   20.84  146.86  -272.96  0.53  2.76  5.02  1023.08

### Calculate Earnings Sharpe Ratio, or alpha adjusted return per unit of risk in the 5 day period after earnings

In [18]:
df = pd.concat(price_list)[['ticker','alpha_5_day']].groupby('ticker').describe().reset_index().sort_values([('alpha_5_day','mean')],ascending=False)
df['earnings_sharpe_ratio'] = df[('alpha_5_day','mean')]/df[('alpha_5_day','std')]
df = df.sort_values([('earnings_sharpe_ratio')],ascending=False)
df['ticker'] = df['ticker']+" US"
df = df.set_axis([f"{x}{y}" for x, y in df.columns], axis=1, inplace=False)[['ticker','earnings_sharpe_ratio']]
df

,ticker,earnings_sharpe_ratio
6,FL US,0.24
0,AEO US,0.23
2,BKE US,0.16
12,ROST US,0.16
7,GES US,0.15
3,BURL US,0.12
14,TPR US,0.09
10,LULU US,0.09
13,TJX US,0.08
4,CPRI US,0.07


### Create a modelset of our stock list

In [23]:
target_modelset = cd.ModelSet(ticker_list = target_list, config=config,allow_nulls=True)

In [40]:
#filter out stuff like EPS and StockPrice and FX
feature_list = target_modelset.common_time_series_names()
feature_list = [item for item in feature_list if 'EPS' not in item]
feature_list = [item for item in feature_list if 'StockPrice' not in item]
feature_list = [item for item in feature_list if 'FX' not in item]

### Create a model_frame (Pandas dataframe) of common names

In [41]:
df_target = target_modelset.model_frame(time_series_name=feature_list,
                                        period_duration_type='fiscal_quarter')

### Create a featureset of model frame and target vector

In [46]:
df_featureset = df_target.pivot(index = ['ticker','period_name'],columns = 'time_series_name',values = 'value')
df_featureset = df_featureset.fillna(0).reset_index()
df_featureset = pd.merge(df_featureset,df,how='inner',left_on='ticker',right_on='ticker')
cols = list(df_featureset.columns)
cols.remove('ticker')
cols.remove('period_name')
df_featureset_r = df_featureset[cols]
df_featureset_r['earnings_sharpe_ratio'] = df_featureset_r['earnings_sharpe_ratio'].astype(float)
cols.remove("earnings_sharpe_ratio")
df_featureset_r.head()

,MO_BSS_Cash,MO_BSS_Debt,MO_BSS_Debt_LT,MO_BSS_Debt_Net,MO_BSS_Debt_ST,MO_BSS_Debt_ToCF,MO_BSS_Debt_ToEBITDA,MO_BSS_IE,MO_BSS_IE_Net,MO_BSS_II,...,MO_VA_EVCalc_NCI,MO_VA_EVCalc_Other,MO_VA_EVCalc_Prefs,MO_VA_EV_ToEBITDA,MO_VA_FCFYield_ToEV,MO_VA_FCFYield_ToMktCap,MO_VA_MarketCap,MO_VA_P_ToCF,MO_VA_P_ToE,earnings_sharpe_ratio
0,496216000.00,0.00,0.00,-496216000.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,4025833870.00,0.00,0.00,0.23
1,327699000.00,0.00,0.00,-327699000.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,2511655800.00,0.00,0.00,0.23
2,326907000.00,0.00,0.00,-326907000.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,3120368400.00,0.00,0.00,0.23
3,238976000.00,0.00,0.00,-238976000.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,3051222360.00,0.00,0.00,0.23
4,225197000.00,0.00,0.00,-225197000.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,2627972270.00,0.00,0.00,0.23


### Run multivariate regression and view fit

In [44]:
import statsmodels.formula.api as smf
formula_string = "+".join(cols)
formula_string = "earnings_sharpe_ratio ~ "+formula_string
mod = smf.ols(formula=formula_string, data=df_featureset_r)
res = mod.fit()
res.summary().tables[0]

Dep. Variable:,earnings_sharpe_ratio,R-squared:,0.881
Model:,OLS,Adj. R-squared:,0.864
Method:,Least Squares,F-statistic:,51.41
Date:,"Sat, 26 Mar 2022",Prob (F-statistic):,3.61e-243
Time:,11:42:09,Log-Likelihood:,1309.3
No. Observations:,740,AIC:,-2431.
Df Residuals:,646,BIC:,-1998.
Df Model:,93,,
Covariance Type:,nonrobust,,


### Display rank ordered coefficients

In [45]:
pd.set_option('display.max_rows', None)
df_out = pd.DataFrame(res.params).reset_index()
df_out.columns = ['time_series_name','Coefficient']
df_out = df_out.sort_values('Coefficient',ascending = False)
pd.merge(df_out,df_target[['time_series_name','time_series_description']],how='inner',left_on='time_series_name',right_on='time_series_name').groupby('time_series_name').first().reset_index().sort_values('Coefficient',ascending=False)

,time_series_name,Coefficient,time_series_description
74,MO_KPI_FranchiseStorePerc,0.01,"Percentage Franchised stores, %"
50,MO_CFSum_DPS,0.01,Dividend Per Share
91,MO_MA_GM,0.00,"Gross Margin (Excluding D&A), %"
90,MO_MA_COGS,0.00,"COGS Margin (Excluding D&A), %"
78,MO_KPI_NetNewStores,0.00,"Net New Stores, # of stores"
86,MO_KPI_Stores_CompanyOwned,0.00,"Total Company Owned Stores, # of stores"
59,MO_CFSum_NetShares_Price,0.00,"Estimated Share Price for Issuance/Buybacks, USD"
13,MO_BSS_NetInterestRate_Debt,0.00,Effective Net Interest Rate on Debt
75,MO_KPI_InventorySpread,0.00,"Sales and Inventory Spread, %"
65,MO_KPI_DIO,0.00,"Days Inventory Outstanding, # of days"
